# AI-Powered Twitter Post Generator

Generates unique tweets using OpenAI API and tracks them in CSV to avoid duplicates.

In [1]:
import tweepy
import os
import csv
import random
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()

# Twitter API credentials
BEARER_TOKEN = os.getenv('BEARER_TOKEN')
API_KEY = os.getenv('API_KEY')
API_SECRET = os.getenv('API_SECRET')
ACCESS_TOKEN = os.getenv('ACCESS_TOKEN')
ACCESS_TOKEN_SECRET = os.getenv('ACCESS_TOKEN_SECRET')

# OpenAI API key
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# CSV file to track posts
CSV_FILE = 'post_history.csv'

In [2]:
# Initialize Twitter API client
client = tweepy.Client(
    bearer_token=BEARER_TOKEN,
    consumer_key=API_KEY,
    consumer_secret=API_SECRET,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET,
    wait_on_rate_limit=True
)

# Initialize OpenAI client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
# Topics for tweet generation
TOPICS = [
    'Artificial Intelligence',
    'Generative AI',
    'Machine Learning',
    'Python Programming',
    'Data Science',
    'Deep Learning',
    'Natural Language Processing',
    'Computer Vision',
    'Neural Networks',
    'Large Language Models',
    'Data Analytics',
    'Big Data',
    'AI Ethics',
    'MLOps',
    'Javascript',
]

In [4]:
def load_post_history():
    """Load previously posted tweets from CSV"""
    try:
        with open(CSV_FILE, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            return [row['text'] for row in reader]
    except FileNotFoundError:
        return []

In [5]:
def save_post(topic, text):
    """Save posted tweet to CSV"""
    file_exists = os.path.isfile(CSV_FILE)
    
    with open(CSV_FILE, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['timestamp', 'topic', 'text'])
        if not file_exists:
            writer.writeheader()
        writer.writerow({
            'timestamp': datetime.now().isoformat(),
            'topic': topic,
            'text': text
        })

In [6]:
def generate_tweet(topic, previous_posts):
    """Generate a unique tweet using OpenAI API"""
    prompt = f"""Generate a single engaging tweet as a question or coding challenge about {topic}. 
    
Requirements:
- Keep it under 280 characters
- Ask a thought-provoking or fun question related to {topic}
- OR share a coding snippet/challenge (like "n % 2 ? 'Odd' : 'Even'", lambda functions, one-liners, etc.)
- Add humor and wit
- Use relevant hashtags (1-2 max)
- Make it educational but entertaining
- Examples: "What's your favorite Python one-liner?", "Can you explain this in plain English: [code snippet]?", "Hot take: [controversial opinion about {topic}]?"
- Don't repeat these previous posts: {previous_posts[-5:] if previous_posts else 'None'}

Just return the tweet text, nothing else."""
    
    response = openai_client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content.strip()

In [7]:
def post_ai_tweet():
    """Generate and post a unique AI-related tweet"""
    # Load previous posts
    previous_posts = load_post_history()
    
    # Select random topic
    topic = random.choice(TOPICS)
    print(f"📝 Generating tweet about: {topic}")
    
    # Generate tweet
    tweet_text = generate_tweet(topic, previous_posts)
    print(f"\n📄 Generated tweet:\n{tweet_text}\n")
    
    # Check if already posted
    if tweet_text in previous_posts:
        print("⚠️ This tweet was already posted. Generating a new one...")
        return post_ai_tweet()
    
    # Post tweet
    try:
        response = client.create_tweet(text=tweet_text)
        print(f"✅ Tweet posted! ID: {response.data['id']}")
        
        # Save to CSV
        save_post(topic, tweet_text)
        print(f"💾 Saved to {CSV_FILE}")
        
        return response.data
    except Exception as e:
        print(f"❌ Error posting tweet: {e}")
        return None

In [8]:
# Generate and post a tweet
post_ai_tweet()

📝 Generating tweet about: Javascript

📄 Generated tweet:
What’s your take: is `const` just the overachieving sibling of `let`? Or is it the guardian angel preventing our code from turning into spaghetti? 🍝👼 #JavaScript #CodeThoughts

✅ Tweet posted! ID: 1991349912727089651
💾 Saved to post_history.csv


{'edit_history_tweet_ids': ['1991349912727089651'],
 'id': '1991349912727089651',
 'text': 'What’s your take: is `const` just the overachieving sibling of `let`? Or is it the guardian angel preventing our code from turning into spaghetti? 🍝👼 #JavaScript #CodeThoughts'}